In [1]:
import sys
import urllib
import json
import requests
import time
import dateutil.parser
from datetime import datetime
# from TDAmeritradeStream import TDAuthentication

from splinter import Browser
from selenium import webdriver

from TDLogin import TDusername, TDpassword, TDconsumerkey

In [12]:
# Assign Working Directory
sys.path.append(r'C:\Users\cinji\Documents\TD Ameritrade\TD Websocket')
sys.path.append(r'C:\Users\cinji\AppData\Local\Programs\Python\Python37\Lib\site-packages')

In [3]:
# Open Chrome Browser

# Define path to Chrome Driver
executable_path = {'executable_path':r'C:\Users\cinji\Desktop\chromedriver_win32\chromedriver'}

# Set some default browser behaviors
options = webdriver.ChromeOptions()

# Make sure the window is maximized
options.add_argument("--start-maximized")

# Create a new browser object
browser = Browser('chrome', **executable_path, headless = False, options = options)

In [4]:
# Open TD Ameritrade App Developer Page

# Define the URL components
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_id = TDconsumerkey + '@AMER.OAUTHAP'
payload = {'response_type':'code', 'redirect_uri':'http://localhost', 'client_id':client_id}

# Build URL
completeURL = requests.Request(method, url, params = payload).prepare()
completeURL = completeURL.url

# Open Log-in page
browser.visit(completeURL)

# Defin the elements to pass through log-in form
payload = {'username':TDusername, 'password':TDpassword}

# Fill out each element in the log-in form
browser.find_by_id("username").first.fill(payload['username'])
browser.find_by_id("password").first.fill(payload['password'])
browser.find_by_id("accept").first.click()

# Click to accept terms and conditions on Authorization page
browser.find_by_id("accept").first.click()

# Provide page loading time
time.sleep(1)
newURL = browser.url

In [5]:
# Get browser URL and parse it to obtain access token
parseURL = urllib.parse.unquote(newURL.split('code=')[1])

# Close browser
browser.quit()

In [6]:
# Authentication Endpoint
# NOTE: This cell can ONLY BE DONE ONCE.  Otherwise, parseURL becomes invalid.

# Define access token endpoint
url = r'https://api.tdameritrade.com/v1/oauth2/token'

# Define access token headers
headers = {'Content-Type':"application/x-www-form-urlencoded"}

# Define access token payload
payload = {'grant_type':'authorization_code',
           'access_type':'offline',
           'code':parseURL,
           'client_id':client_id,
           'redirect_uri':'http://localhost'}

# Post data to get access token
authReply = requests.post(url, headers = headers, data = payload)

# Convert JSON string to dictionary
decodedURL = authReply.json()

In [7]:
# Parse retrieved access token
AccessToken = decodedURL['access_token']
RefreshToken = decodedURL['refresh_token']
headers = {'Authorization': "Bearer {}".format(AccessToken)}

In [8]:
import datetime

def unix_time_millis(dt):
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0

In [9]:
# Define endpoint for User Principals
endpoint = 'https://api.tdameritrade.com/v1/userprincipals'

headers = {'Authorization':'Bearer {}'.format(AccessToken)}

# Define endpoint parameters
params = {'fields': 'streamerSubscriptionKeys,streamerConnectionInfo'}

# Make a request
content = requests.get(url = endpoint, params = params, headers = headers)
userPrincipalsResponse = content.json()

# Convert timestamp to milliseconds
tokenTimeStamp = userPrincipalsResponse['streamerInfo']['tokenTimestamp']
date = dateutil.parser.parse(tokenTimeStamp, ignoretz = True)
tokenTimeStampAsMs = unix_time_millis(date)
tokenTimeStampAsMs
# tokenTimeStampAsMs

# Define needed items in order to make Login request
credentials = {'userid':userPrincipalsResponse['accounts'][0]['accountId'],
               'token':userPrincipalsResponse['streamerInfo']['token'],
               'company':userPrincipalsResponse['accounts'][0]['company'],
               'segment':userPrincipalsResponse['accounts'][0]['segment'],
               'cddomain':userPrincipalsResponse['accounts'][0]['accountCdDomainId'],
               'usergroup':userPrincipalsResponse['streamerInfo']['userGroup'],
               'accesslevel':userPrincipalsResponse['streamerInfo']['accessLevel'],
               'authorized':"Y",
               'timestamp':int(tokenTimeStampAsMs),
               'appid':userPrincipalsResponse['streamerInfo']['appId'],
               'acl':userPrincipalsResponse['streamerInfo']['acl']}

In [10]:
# Define login request
login_request = {"requests": [{"service": "ADMIN",
                               "command": "LOGIN",
                               "requested": 0,
                               "account": userPrincipalsResponse['accounts'][0]['accountId'],
                               "source": userPrincipalsResponse['streamerInfo']['appId'],
                               "parameters": {"credential": urllib.parse.urlencode(credentials),
                                              "token": userPrincipalsResponse['streamerInfo']['token'],
                                              "version": "1.0"}}]}

# Define requests for different data sources
data_request = {"requests": [
    {
        "service": "ACTIVES_NASDAQ",
        "requestid": "1",
        "command": "SUBS",
        "account": userPrincipalsResponse['accounts'][0]['accountId'],
        "source": userPrincipalsResponse['streamerInfo']['appId'],
        "parameters": {
            "keys": "NASDAQ-60",
            "fields": "0,1"
        }
    },
    {
        "service": "LEVELONE_FUTURES",
        "requestid": "2",
        "command": "SUBS",
        "account": userPrincipalsResponse['accounts'][0]['accountId'],
        "source": userPrincipalsResponse['streamerInfo']['appId'],
        "parameters": {
            "keys": "/ES",
            "fields": "0,1,2,3,4"
        }
    }
]}

# Turn requests into JSON String
login_encoded = json.dumps(login_request)
data_encoded = json.dumps(data_request)
data_encoded

'{"requests": [{"service": "ACTIVES_NASDAQ", "requestid": "1", "command": "SUBS", "account": "861370305", "source": "Cinji1218", "parameters": {"keys": "NASDAQ-60", "fields": "0,1"}}, {"service": "LEVELONE_FUTURES", "requestid": "2", "command": "SUBS", "account": "861370305", "source": "Cinji1218", "parameters": {"keys": "/ES", "fields": "0,1,2,3,4"}}]}'

In [22]:
import websockets
import asyncio
import pyodbc

class WebSocketClient():
    
    def __init__(self):
        self.cnxn = None
        self.crsr = None
        
    def database_connect(self):
        
        # Define server and database
        server = 'DESKTOP-O9QIKO1\SQLEXPRESS'
        database = 'TD Ameritrade Stock Database'
        sql_driver = '{ODBC Driver 17 for SQL Server}'
        
        # Define database connection
        self.cnxn = pyodbc.connect(driver = sql_driver,
                                   server = server,
                                   database = database,
                                   trusted_connection = 'yes')
        
        self.crsr = self.cnxn.cursor()
        
    def database_insert(self, query, data_tuple):
        
        self.crsr.execute(query, data_tuple)
        self.cnxn.commit()
        self.cnxn.close()
        print('Data has been successfully inserted into the database')
        
    async def connect(self):
        
        uri = "wss://" + userPrincipalsResponse['streamerInfo']['streamerSocketUrl'] + "/ws"
        
        self.connection = await websockets.client.connect(uri)
        
        if self.connection.open:
            print("Connection established. Client correctly connected.")
            return self.connection
        
    async def sendMessage(self, message):
        await self.connection.send(message)
        
    async def receiveMessage(self, connection):
        
        while True:
            
            try:
                
                # Decode message
                message = await connection.recv()
                message_decoded = json.loads(message)
                
                # Prepare data for insertion, connect to database
                query = "INSERT INTO  StockData (service, timestamp, command) VALUES (?,?,?);"
                self.database_connect()
                
                if 'data' in message_decoded.keys():
                    
                    # Grab data
                    data = message_decoded['data'][0]
                    data_tuple = (data['service'], data['timestamp'], data['command'])
                    
                    # Insert data
                    self.database_insert(query, data_tuple)
                    
                print('-'*20)
                print('Received message from server ' + str(message))
                    
            except websockets.exceptions.ConnectionClosed:
                print('Connection with server closed')
                break
                    
    async def heartbeat(self, connection):
        
        while True:
            try:
                await connection.send('ping')
                await aysncio.sleep(5)
                
            except websockets.exceptions.ConnectionClosed:
                print('Connection with server closed')
                break

In [23]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == '__main__':
    
    # Create client object
    client = WebSocketClient()
    
    # Define event loop
    loop = asyncio.get_event_loop()
    
    # Start a connection to the websocket
    connection = loop.run_until_complete(client.connect())
    
    # Define tasks to run
    tasks = [asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(login_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection)),
             asyncio.ensure_future(client.sendMessage(data_encoded)),
             asyncio.ensure_future(client.receiveMessage(connection))]
    
    # Run tasks
    loop.run_until_complete(asyncio.wait(tasks))

Connection established. Client correctly connected.


In [17]:
client

In [18]:
client.connect()

<coroutine object WebSocketClient.connect at 0x0000012B8DFAB8C8>

In [19]:
loop

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [20]:
loop.run_until_complete(client.connect())

Connection established. Client correctly connected.
